# SLAKH Dataset Working

In [1]:
import os
import yaml
from benedict import benedict
import matplotlib.pyplot as plt
import torch
import mirdata
import time

%load_ext autoreload
%autoreload 2

from use_openunmix import Target, Track, SlakhDataset

In [2]:
slakh_dataset = SlakhDataset()

# 0.036s for loading of stem audio
# 0.097s for loading of mix audio

  0%|          | 0/1289 [00:00<?, ?it/s]

In [14]:
slakh_dataset.target = "Trumpet"
slakh_dataset.filter_target()
print(slakh_dataset.tracks[0].targets[0].instrument)
# print(slakh_dataset.targets)

Fretless Bass


In [47]:
for i in range(5):
    slakh_dataset[0][0].shape

In [ ]:
for i in range(len(slakh_dataset.tracks)):
    print(slakh_dataset.tracks[i]," -> ",[slakh_dataset.tracks[i].targets[j].instrument for j in range(len(slakh_dataset.tracks[i].targets))])

In [48]:
# des_targets = set()
# des_tracks = set()
# for tr,track in enumerate(slakh_dataset.all_tracks):
#     for ta, target in enumerate(track.targets):
#         if target.instrument == 'French Horn':
#             des_targets.add(target)
#             des_tracks.add(track)
# print(list(des_targets))
# print(list(des_tracks))

## Functions helpful for filtering the dataset

In [25]:
DATASET_FOLDER = "D:/03_Dokumente/01_epfl/01_cm/slakh2100_flac_redux/train"

track_folders = sorted(os.listdir(DATASET_FOLDER))
stem_audio_paths = {}
mix_audio_paths = {}

instruments = []
instruments_folder = []

for track_folder in track_folders:
    d = benedict.from_yaml(DATASET_FOLDER + "/"+ track_folder + "/metadata.yaml")
    instruments_stem = []
    mix_audio_paths[track_folder[-5:]] = DATASET_FOLDER + "/" + track_folder + "/mix.wav"
    stems = {}
    for stem in sorted(d["stems"].keys()):
        stems[stem] = DATASET_FOLDER + "/" + track_folder + "/stems/" + stem + ".wav"
        instrument = d["stems"][stem]["midi_program_name"]
        instruments.append(instrument)
        instruments_stem.append(instrument)

    # print(track_folder,instruments_stem)
    instruments_folder.append(instruments_stem)
    stem_audio_paths[track_folder[-5:]] = stems

In [26]:
instruments

['Distortion Guitar',
 'Drums',
 'Bright Acoustic Piano',
 'Electric Bass (finger)',
 'Choir Aahs',
 'Percussive Organ',
 'FX 8 (sci-fi)',
 'Electric Guitar (jazz)',
 'Distortion Guitar',
 'Harmonica',
 'Harmonica',
 'Synth Strings 1',
 'Electric Bass (pick)',
 'Lead 2 (sawtooth)',
 'Pad 1 (new age)',
 'Choir Aahs',
 'Music Box',
 'Clavinet',
 'Brass Section',
 'Electric Guitar (muted)',
 'Drums',
 'Woodblock',
 'Orchestra Hit',
 'Bright Acoustic Piano',
 'Electric Bass (pick)',
 'Acoustic Guitar (steel)',
 'Overdriven Guitar',
 'Electric Guitar (jazz)',
 'Fiddle',
 'Drums',
 'Electric Guitar (clean)',
 'Acoustic Guitar (steel)',
 'Pad 3 (polysynth)',
 'Electric Bass (finger)',
 'Choir Aahs',
 'Rock Organ',
 'Electric Guitar (muted)',
 'Synth Strings 1',
 'Harpsichord',
 'Acoustic Grand Piano',
 'Drums',
 'Drums',
 'Acoustic Guitar (steel)',
 'Acoustic Grand Piano',
 'Pan Flute',
 'Electric Bass (finger)',
 'Glockenspiel',
 'Electric Piano 2',
 'Flute',
 'Overdriven Guitar',
 'FX 4 (at

In [ ]:
# print(sorted(set(instruments)))
interesting_instrs = ['French Horn', 'Bassoon','Clarinet','Electric Guitar (clean)','Viola', 'Violin', 'Trombone', 'Trumpet', 'Tuba','Alto Sax','Tenor Sax']
indxs = [[] for i in range(len(interesting_instrs))]
interesting_instrs_indxs = dict(zip(interesting_instrs, indxs))
for inst in interesting_instrs:
    interesting_instrs_indxs[inst] = [i for i, x in enumerate(instruments) if x == inst]
    print(inst,len(interesting_instrs_indxs[inst]))
instruments_folder

In [ ]:
print(set(instruments))
plt.hist(instruments)

In [ ]:
dataset = mirdata.initialize('slakh',"../datasets")
slakh_data = dataset.load_tracks()
print(mirdata.list_datasets())
print(dataset.remotes)
# dataset.download(['baby'])
print(dataset.get_mtrack_splits()['test'][:3])
print(dataset.get_track_splits()['train'][:3])
slakh_data['Track00001-S01'].instrument

In [ ]:
slakh_data = dataset.load_tracks()
tic = time.perf_counter()
for i in range(1000):
    x,sr = mirdata.datasets.slakh.load_audio(stem_audio_paths["00001"]["S00"])
toc = time.perf_counter()
print(f"{toc-tic:.4}")
plt.plot(x)

In [7]:
import os
import yaml
from benedict import benedict
import matplotlib.pyplot as plt
import time
import shutil

In [27]:
DATASET_FOLDER = "D:/03_Dokumente/01_epfl/01_cm/slakh2100_flac_redux"

INSTRUMET = "Electric Guitar (jazz)"
NEW_DATASET_FOLDER = "D:/03_Dokumente/01_epfl/01_cm/slakh2100_flac_redux_guitar_jazz"

splits = ["train","test","validation"]

for split in splits:

    #if not os.path.exists(DATASET_FOLDER+"/"+split):
    os.mkdir(f"{NEW_DATASET_FOLDER}/{split}")
    
    track_folders = sorted(os.listdir(DATASET_FOLDER+"/"+split))
    for track_folder in track_folders[:]:

        d = benedict.from_yaml(f"{DATASET_FOLDER}/{split}/{track_folder}/metadata.yaml")
        
        
        for stem in sorted(d["stems"].keys()):
            stem_folder = f"{DATASET_FOLDER}/{split}/{track_folder}/stems"
            new_stem_folder = f"{NEW_DATASET_FOLDER}/{split}/{track_folder}/stems"
            instrument = d["stems"][stem]["midi_program_name"]
            if INSTRUMET == instrument:
                if not os.path.exists(f"{NEW_DATASET_FOLDER}/{split}/{track_folder}"):
                    os.mkdir(f"{NEW_DATASET_FOLDER}/{split}/{track_folder}")
                
                shutil.copyfile(f"{DATASET_FOLDER}/{split}/{track_folder}/metadata.yaml", f"{NEW_DATASET_FOLDER}/{split}/{track_folder}/metadata.yaml") 

                shutil.copyfile(f"{DATASET_FOLDER}/{split}/{track_folder}/mix.flac", f"{NEW_DATASET_FOLDER}/{split}/{track_folder}/mix.flac")
                if not os.path.exists(new_stem_folder):
                    os.mkdir(f"{new_stem_folder}")
                try:
                    shutil.copyfile(f"{stem_folder}/{stem}.flac", f"{new_stem_folder}/{stem}.flac")
                except:
                    print("Did not work for ", split, track_folder,stem)


Did not work for  train Track00164 S05
Did not work for  train Track01216 S07
Did not work for  train Track01216 S10
Did not work for  train Track01216 S11
Did not work for  train Track01216 S12
Did not work for  train Track01216 S13
Did not work for  validation Track01720 S05


In [4]:
True if 1 else False

True